# urllib模块使用

## 1、urlopen()方法

In [1]:
import urllib.request

response = urllib.request.urlopen('https://www.python.org')
# print(response.read().decode('utf-8'))

# 返回结果
print('返回类型:', type(response))
print('响应状态:', response.status)
print('响应头:', response.getheaders())
print('响应服务器:', response.getheader('Server'))

返回类型: <class 'http.client.HTTPResponse'>
响应状态: 200
响应头: [('Server', 'nginx'), ('Content-Type', 'text/html; charset=utf-8'), ('X-Frame-Options', 'DENY'), ('Via', '1.1 vegur'), ('Via', '1.1 varnish'), ('Content-Length', '49130'), ('Accept-Ranges', 'bytes'), ('Date', 'Sun, 05 May 2019 14:04:13 GMT'), ('Via', '1.1 varnish'), ('Age', '2473'), ('Connection', 'close'), ('X-Served-By', 'cache-iad2131-IAD, cache-hnd18749-HND'), ('X-Cache', 'HIT, HIT'), ('X-Cache-Hits', '1, 1886'), ('X-Timer', 'S1557065053.091904,VS0,VE0'), ('Vary', 'Cookie'), ('Strict-Transport-Security', 'max-age=63072000; includeSubDomains')]
响应服务器: nginx


**urlopen()方法API： urlopen(url, data=None, [timeout, ]*, cafile=None, capath=None, cadefaule=False, context=None)** <br>
- **data**： 可选参数，bytes类型，使用该参数则请求方式为POST<br>
- **timeout**: 可选参数，设置超时时间，若超出该时间，则抛出异常。<br>
- **cafile, capath**：分别指定CA证书和它的路径<br>
- **context**: 必须是ssl.SSLContext类型，指定SSL设置<br>
- **cadefault**: 该参数已经弃用，默认值为False<br>

In [4]:
import urllib.request

# data参数使用
data = bytes(urllib.parse.urlencode({'world': 'hello'}), encoding='utf8')
response = urllib.request.urlopen('http://httpbin.org/post', data=data)
print(response.read().decode('utf-8'))

{
  "args": {}, 
  "data": "", 
  "files": {}, 
  "form": {
    "world": "hello"
  }, 
  "headers": {
    "Accept-Encoding": "identity", 
    "Content-Length": "11", 
    "Content-Type": "application/x-www-form-urlencoded", 
    "Host": "httpbin.org", 
    "User-Agent": "Python-urllib/3.5"
  }, 
  "json": null, 
  "origin": "58.19.2.46, 58.19.2.46", 
  "url": "https://httpbin.org/post"
}



In [6]:
import urllib.request
import socket

# timeout参数
try:
    response = urllib.request.urlopen('http://httpbin.org/get', timeout=0.1)
except urllib.error.URLError as e:
    if isinstance(e.reason, socket.timeout):
        print('Time Out')

Time Out


# 2、urllib.request.Request类使用

使用Request类型的对象，将请求独立成一个对象，更加丰富灵活地配置参数

In [2]:
# 基础用法
import urllib.request

request = urllib.request.Request('https://python.org')
response = urllib.request.urlopen(request)
# print(response.read().decode('utf-8'))

**Request()API： Requset(url, data=None, headers={}, origin_req_host=None, unverifiable=False, method=None)**<br>
- **url**: 请求URL，必传参数<br>
- **data**: bytes类型，bytes(urllib.parse.urlencode(dict))<br>
- **headers**: 请求头，直接构造或者使用add_header()方法添加<br>
- **origin_req_host**: 请求方的host名称或者IP地址<br>
- **unverifiable**: 请求是否是无法验证<br>
- **method**: 请求方法，GET、POST等等<br>

In [5]:
# 传入多个参数
from urllib import request, parse

url = 'http://httpbin.org/post'
# User-Agent: 用于伪装浏览器
headers = {
    'User-Agent': 'Mozilla/4.0 (compatible; MSIE 5.5; Windows NT)',
    'Host': 'httpbin.org'
}
dict = {
    'name': 'Getmey'
}
data = bytes(parse.urlencode(dict), encoding='utf-8')
req = request.Request(url=url, data=data, headers=headers, method='POST')
response = request.urlopen(req)
print(response.read().decode('utf-8'))

{
  "args": {}, 
  "data": "", 
  "files": {}, 
  "form": {
    "name": "Getmey"
  }, 
  "headers": {
    "Accept-Encoding": "identity", 
    "Content-Length": "11", 
    "Content-Type": "application/x-www-form-urlencoded", 
    "Host": "httpbin.org", 
    "User-Agent": "Mozilla/4.0 (compatible; MSIE 5.5; Windows NT)"
  }, 
  "json": null, 
  "origin": "218.197.142.9, 218.197.142.9", 
  "url": "https://httpbin.org/post"
}



# 3、高级用法

各种各样的Hadler类，分别能处理登录验证，Cookies,代理设置等等。<br>
urllib.request中的**BaseHandler**类为其他Handler的父类。<br>
- **HTTPDefaultErrorHandler**: 用于处理HTTP相应错误，错误都会抛出HTTPError类型的异常。<br>
- **HTTPRedirectHandler**: 用于处理重定向。<br>
- **HTTPCookieProcessor**: 用于处理Cookies。<br>
- **ProxyHandler**: 用于设置代理，默认代理为空。<br>
- **HTTPPasswordMgr**: 用于管理密码，维护用户名和密码的表。<br>
- **HTTPBasicAuthHandler**: 用于管理验证，例如打开链接时需要账号密码登录等。<br>

## 3.1 验证

![image](https://github.com/DRNTT/SpiderImage/blob/master/ch3/verify.png?raw=true)

In [8]:
# 借助HTTPBasicAuthHandler完成
from urllib.request import HTTPPasswordMgrWithDefaultRealm, HTTPBasicAuthHandler, build_opener
from urllib.error import URLError

uesrname = 'username'
password = 'password'
url = 'http://localhost:5000/'

p = HTTPPasswordMgrWithDefaultRealm()
p.add_password(None, url, username, password)
auth_handler = HTTPBasicAuthHandler(p)
opener = build_opener(auth_handler)

try:
    result = opener.open(url)
    html = result.read().decode('utf-8')
    print(html)
except URLError as e:
    print(e.reason)

## 3.2 代理

方法1：在本地搭建一个代理，运行在9743端口上。(该方法需要在本地974端口上搭建HTTP服务才可使用）<br>
方法2：在网站https://www.xicidaili.com/nn/ 找到其他代理使用。（注意是HTTP协议还是HTTPS协议的代理，同时修改端口）

In [17]:
from urllib.error import URLError
from urllib.request import ProxyHandler, build_opener

proxy_handler = ProxyHandler({
    'http': 'http://119.102.128.231:9999',
    'https': 'https://222.135.92.68:38094'
})
opener = build_opener(proxy_handler)
try:
    response = opener.open('https://www.baidu.com')
    print(response.read().decode('utf-8'))
except URLError as e:
    print(e.reason)

<html>
<head>
	<script>
		location.replace(location.href.replace("https://","http://"));
	</script>
</head>
<body>
	<noscript><meta http-equiv="refresh" content="0;url=http://www.baidu.com/"></noscript>
</body>
</html>


## 3.3 Cookies

截取网站的Cookies。

In [20]:
import http.cookiejar, urllib.request

# 先声明一个CookieJar对象，利用HTTPCookieProcessor构建一个Handler
# 再使用build_open()方法构建opener，执行open()函数。
cookie = http.cookiejar.CookieJar()
handler = urllib.request.HTTPCookieProcessor(cookie)
opener = urllib.request.build_opener(handler)

response = opener.open('http://www.baidu.com')
for item in cookie:
    print(item.name + "=" + item.value)

BAIDUID=DC312294DA755861379ADA54B45EDB59:FG=1
BIDUPSID=DC312294DA755861379ADA54B45EDB59
H_PS_PSSID=1464_21112_28772_28720_28963_28833_28584_26350_28603
PSTM=1557054637
delPer=0
BDSVRTM=0
BD_HOME=0


将截取的Cookies保存为文件。

In [21]:
import http.cookiejar, urllib.request

filename = 'cookies.txt'
cookie = http.cookiejar.MozillaCookieJar(filename)
handler = urllib.request.HTTPCookieProcessor(cookie)
opener = urllib.request.build_opener(handler)

requese = opener.open('http://www.baidu.com')
# ignore_discard的意思是即使cookies将被丢弃也将它保存下来
# ignore_expires的意思是如果在该文件中cookies已经存在，则覆盖原文件写入
cookie.save(ignore_discard=True, ignore_expires=True)

![image](https://github.com/DRNTT/SpiderImage/blob/master/ch3/cookie.png?raw=true)

In [22]:
# LWP格式的Cookies文件
import http.cookiejar, urllib.request

filename = 'LWPcookies.txt'
cookie = http.cookiejar.LWPCookieJar(filename)
handler = urllib.request.HTTPCookieProcessor(cookie)
opener = urllib.request.build_opener(handler)

requese = opener.open('http://www.baidu.com')
# ignore_discard的意思是即使cookies将被丢弃也将它保存下来
# ignore_expires的意思是如果在该文件中cookies已经存在，则覆盖原文件写入
cookie.save(ignore_discard=True, ignore_expires=True)

![image](https://github.com/DRNTT/SpiderImage/blob/master/ch3/LWPcookies.png?raw=true)

读取并利用保存后的Cookies文件。<br>
先前直接open百度的url时，返回的并非baidu的真正html文件内容，我们需要设置好User-Agent才能获取到。

In [3]:
import http.cookiejar, urllib.request

cookie = http.cookiejar.LWPCookieJar()
cookie.load('LWPcookies.txt', ignore_discard=True, ignore_expires=True)
handler = urllib.request.HTTPCookieProcessor(cookie)
opener = urllib.request.build_opener(handler)

header = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/74.0.3729.131 Safari/537.36'
}
request = urllib.request.Request('https://www.baidu.com', headers=header)

response = opener.open(request)
# print(response.read().decode('utf-8'))

# 4 异常处理

## 4.1、URLError

URLError集成OSError，是error异常模块的基类，由request模块生的异常都可以通过捕捉这个类进行处理。

In [35]:
from urllib import request, error
try:
    response = request.urlopen('https://cuiqingcai.com/index.htm')
except error.URLError as e:
    print(e.reason)

Not Found


## 4.2 HTTPError

HTTPError专门用来处理HTTP请求错误。属性如下：
- **code**: 返回HTTP状态码
- **reason**: 同父类，返回错误原因
- **headers**: 返回请求头

In [40]:
from urllib import request, error
try:
    response = request.urlopen("https://cuiqingcai.com/index.htm")
except error.HTTPError as e:
    print(e.reason, e.code, e.headers, sep='\n')
except error.URLError as e:
    print(e.reason)
else:
    print('Request Successfully')

Not Found
404
Server: nginx/1.10.3 (Ubuntu)
Date: Sun, 05 May 2019 11:50:58 GMT
Content-Type: text/html; charset=UTF-8
Transfer-Encoding: chunked
Connection: close
Vary: Cookie
Expires: Wed, 11 Jan 1984 05:00:00 GMT
Cache-Control: no-cache, must-revalidate, max-age=0
Link: <https://cuiqingcai.com/wp-json/>; rel="https://api.w.org/"




## 4.3 reason属性的其他返回类型

In [42]:
import socket
import urllib.request
import urllib.error

try:
    reponse = urllib.request.urlopen("https://www.baidu.com", timeout=0.01)
except urllib.error.URLError as e:
    print(type(e.reason))
    if isinstance(e.reason, socket.timeout):
        print('TIME OUT')

<class 'socket.timeout'>
TIME OUT


# 5、解析链接

## 5.1 urlparse(): 实现URL的识别和分段

In [44]:
from urllib.parse import urlparse

result = urlparse("http://www.baidu.com/index.html;user?id=5#comment")
print(type(result), result, sep='\n')

<class 'urllib.parse.ParseResult'>
ParseResult(scheme='http', netloc='www.baidu.com', path='/index.html', params='user', query='id=5', fragment='comment')


result为ParseResult对象，共有6个部分。
用结果映射网站地址即为：scheme://netloc/path;params?query#fragment
- **scheme**: 协议
- **netloc**: 域名
- **path**: 访问路径
- **params**: 参数
- **query**: 查询条件
- **fragment**： 锚点，定位页面内部的下拉位置

具体API用法：**ulrparse(urlstring,, scheme='', allow_fragments=True)
- **urlstring**: 必填，待解析url
- **scheme**: 默认协议，若urlstring中没有带有协议，则将此作为默认协议
- **allow_fragments**: 若为False,则结果中fragment部分被忽略，该部分被解析为path, params或者query的其中一部分。

In [48]:
from urllib.parse import urlparse

# 默认协议
result = urlparse("www.baidu.com/index.html;user?id=5#comment", scheme='https')
print(type(result), result, sep='\n')

# 忽略fragment
result = urlparse("https://www.baidu.com/index.html;user?id=5#comment", allow_fragments=False)
print(type(result), result, sep='\n')

# 通过索引或者属性名获取值
print(result[0], result.scheme, sep='\n')

<class 'urllib.parse.ParseResult'>
ParseResult(scheme='https', netloc='', path='www.baidu.com/index.html', params='user', query='id=5', fragment='comment')
<class 'urllib.parse.ParseResult'>
ParseResult(scheme='https', netloc='www.baidu.com', path='/index.html', params='user', query='id=5#comment', fragment='')
https
https


## 5.2 urlunparse()：与urlparse()方法作用相反

需要填入一个可迭代对象，长度必须为6，否则抛出异常。

In [49]:
from urllib import parse

data = ['https', 'baidu.com', '/index.html', 'user', 'id=7', 'comment']
html = parse.urlunparse(data)
print(html)

https://baidu.com/index.html;user?id=7#comment


## 5.3 urlsplit()与urlunsplit()

urlsplit与urlparse不同的是只解析出5个结果，并且返回类型为SplitResult,不再单独解析出params。

In [51]:
from urllib import parse

data = parse.urlsplit('https://www.baidu.com/index.html;user?id=6#comment')
print(data)
html = parse.urlunsplit(data)
print(html)

SplitResult(scheme='https', netloc='www.baidu.com', path='/index.html;user', query='id=6', fragment='comment')
https://www.baidu.com/index.html;user?id=6#comment


## 5.4 urljoin()

提供一个base_url作为第一个参数，新链接作为第二个参数。该方法解析出base_url中的scheme, netloc, path并对新链接中缺失的部分进行填充，若新链接中已经存在这些部分，则不作替换。

In [57]:
from urllib.parse import urljoin 

# 往新链接中加入base_url中的scheme与netloc
print(urljoin('http://www.baidu.com', 'FAQ.html'))
# 不改动新链接
print(urljoin('http://www.baidu.com', 'https://cuiqingcai.com/FAQ.html'))
# 不改动新链接
print(urljoin('http://www.baidu.com/about.html', 'https://cuiqingcai.com/FAQ.html'))
# 不改动新链接
print(urljoin('http://www.baidu.com/about.html', 'https://cuiqingcai.com/FAQ.html?question=2'))
# 只获取base_url中的scheme, netloc与path
print(urljoin('http://www.baidu.com?wd=abc', 'https://cuiqingcai.com/index.php'))
# 在新链接中加入base_url中的scheme, netloc
print(urljoin('http://www.baidu.com', '?category=2#comment'))
# 在新链接中加入base_url中的netloc
print(urljoin('www.baidu.com', '?category=2#comment'))
# 在新链接中加入base_url中的netloc
print(urljoin('www.baidu.com#comment', '?category=2'))

http://www.baidu.com/FAQ.html
https://cuiqingcai.com/FAQ.html
https://cuiqingcai.com/FAQ.html
https://cuiqingcai.com/FAQ.html?question=2
https://cuiqingcai.com/index.php
http://www.baidu.com?category=2#comment
www.baidu.com?category=2#comment
www.baidu.com?category=2


## 5.5 urlencode()

在构造GET请求参数时非常有用，同样可以构造POST请求参数。

In [64]:
from urllib.parse import urlencode
import urllib.request

# 构造GET请求参数
params = {
    'name': 'germey',
    'age': 22
}
base_url = 'http://www.baidu.com?'
url = base_url + urlencode(params)
print(url)

# 构造POST请求参数
data = bytes(urlencode(params), encoding='utf-8')
response = urllib.request.urlopen('http://httpbin.org/post', data=data)
print(response.read().decode('utf-8'))

http://www.baidu.com?name=germey&age=22
{
  "args": {}, 
  "data": "", 
  "files": {}, 
  "form": {
    "age": "22", 
    "name": "germey"
  }, 
  "headers": {
    "Accept-Encoding": "identity", 
    "Content-Length": "18", 
    "Content-Type": "application/x-www-form-urlencoded", 
    "Host": "httpbin.org", 
    "User-Agent": "Python-urllib/3.5"
  }, 
  "json": null, 
  "origin": "58.19.2.46, 58.19.2.46", 
  "url": "https://httpbin.org/post"
}



## 5.6 parse_qs()与parse_qsl()

- **parse_qs()**: 反序列化，将一串GET请求参数转回字典
- **parse_qsl()**: 反序列化，将一串GET请求参数转位元组组成的列表

In [65]:
from urllib.parse import parse_qs, parse_qsl

query = 'name=laowang&age=30'
print(parse_qs(query), parse_qsl(query), sep='\n')

{'name': ['laowang'], 'age': ['30']}
[('name', 'laowang'), ('age', '30')]


## 5.7 quote()与unquote()

- **quote()**: 将内容转位URL编码格式，避免中文乱码情况。
- **unquote()**: 将URL编码转为中文

In [67]:
from urllib.parse import quote, unquote

keyword = '壁纸'
url = 'https://www.baidu.com/s?wd=' + quote(keyword)
print(url)
print(unquote(url))

https://www.baidu.com/s?wd=%E5%A3%81%E7%BA%B8
https://www.baidu.com/s?wd=壁纸


# 6、Robots协议

Robots协议为爬虫协议、机器人协议，用来告诉爬虫和搜索引擎哪些页面可以抓取，哪些不可以抓取。通常为一个叫做robots.txt，一般存放于网站根目录下。<br>
robots.txt样例<br>
User-agent: * <br>
Disallow: /<br>
Allow: /public/<br>
- **User-agent**: 描述爬虫名称，*为对任何爬虫有效。例如，Baiduspider则为对百度爬虫有效。
- **Disallow**: 指定了不允许爬取的目录，/代表不允许爬去所有页面。
- **Allow**: 一般与Disallow一起使用。/public/，则表示所有页面不允许爬去，除了public目录。

## 6.1 使用robotparser模块解析robots.txt

**常用方法：**
- **set_url()**: 设置robots.txt文件的链接，如果在创建RobotFileParser对象时，传入了链接，则不用使用该方法设置。
- **read()**: 读取robots.txt文件的内容并分析。如果不调用该方法，则接下来的所有判断都为False,并且该方法不返回任何内容，但是进行了读取操作。
- **parse()**: 用来解析robots.txt文件。
- **can_fetch()**: 传入两个参数，一个是User-agent, 另一个为要抓取的URL。返回该搜索引擎是否可以抓取这个URL。
- **mtime()**: 返回上次抓取和分析robots.txt的时间。
- **modified()**: 将当前时间设置为上次抓取和分析robots.txt的时间。

In [71]:
from urllib.robotparser import RobotFileParser

rp = RobotFileParser()
rp.set_url('http://www.jianshu.com/robots.txt')
rp.read()
print(rp.can_fetch('*', 'http://www.jianshu.com/p/b67554025d7d'))
print(rp.can_fetch('*', 'http://www.jianshu.com/search?q=python&page=1&type=collections'))


False
False


In [77]:
from urllib.robotparser import RobotFileParser

# 可以提前进入https://www.baidu.com/robots.txt文件中查看内容，再进行爬取。
rp = RobotFileParser()
rp.set_url('http://www.baidu.com/robots.txt')
rp.read()
print(rp.can_fetch('Baiduspider', 'http://www.baidu.com/robots.txt'))

True


In [4]:
from urllib.request import urlopen

reponse = urlopen('https://www.baidu.com/robots.txt')
# print(reponse.read().decode('utf-8'))